In [1]:
import numpy as np


def gram_schmidt(basis):
    n = len(basis)
    dim = len(basis[0])

    ortho = [np.zeros(dim) for _ in range(n)]
    mu = np.zeros((n, n))

    for i in range(n):
        b_i = basis[i]
        projection = np.zeros(dim)

        for j in range(i):
            mu[i, j] = np.dot(b_i, ortho[j]) / np.dot(ortho[j], ortho[j])
            projection += mu[i, j] * ortho[j]

        ortho[i] = b_i - projection

    return ortho, mu



def lll_reduce(basis, delta=0.75, verbose=False):
    basis = [b.copy() for b in basis]
    n = len(basis)
    k = 1

    while k < n:

        ortho, mu = gram_schmidt(basis)

        for j in range(k - 1, -1, -1):  # j < k

            if abs(mu[k, j]) > 0.5:
                r = round(mu[k, j])
                basis[k] -= r * basis[j]
                ortho, mu = gram_schmidt(basis)

        norm_sq_prev = np.dot(ortho[k - 1], ortho[k - 1])
        norm_sq_curr = np.dot(ortho[k], ortho[k])

        lhs = delta * norm_sq_prev
        rhs = norm_sq_curr + mu[k, k - 1]**2 * norm_sq_prev

        if lhs > rhs:
            basis[k], basis[k - 1] = basis[k - 1], basis[k].copy()

            k = max(k - 1, 1)
        else:
            k += 1


    return basis





In [18]:
import import_ipynb
import basis_reduction_2d as br2d; # type: ignore ## static funcs error?

def bases_equal_2d(basis1, basis2):
    def normalize(v):
        v = np.array(v)
        return v if v[0] > 0 or (v[0] == 0 and v[1] >= 0) else -v

    def sorted_basis(u, v):
        return sorted([normalize(u).tolist(), normalize(v).tolist()])

    return sorted_basis(basis1[0], basis1[1]) == sorted_basis(basis2[0], basis2[1])

def tests_brlll(basis_list, verbose=False):

    tests_amount = len(basis_list)
    tests_passed = 0

    results = []

    for i, (b1, b2) in enumerate(basis_list):
        b1_reduced, b2_reduced = lll_reduce([b1, b2])

        same = br2d.check_same_lattice(b1, b2, b1_reduced, b2_reduced)
        original_len = min(np.linalg.norm(b1), np.linalg.norm(b2))
        reduced_len = min(np.linalg.norm(b1_reduced), np.linalg.norm(b2_reduced))
        improved = reduced_len <= original_len

        if same and improved:
            tests_passed += 1
            if verbose:
                print(f"✅ Test {i + 1}: PASSED")
                print(f"Initial basis: b1 = {b1}, b2 = {b2}")
                print(f"Reduced basis: b1 = {b1_reduced}, b2 = {b2_reduced}")

            results.append({
                    "b1": b1_reduced,
                    "b2": b2_reduced,
                    "result": 1
            })
        else:
            if verbose:
                print(f"❌ Test {i + 1} FAILED: ")
                print(f"b1 = {b1}, b2 = {b2}")

            results.append({
                    "b1": b1_reduced,
                    "b2": b2_reduced,
                    "result": 0
            })

    if verbose:
        print(f"\n📊 {tests_amount}/{tests_passed} tests passed.")
    return results



In [17]:
basis_list = br2d.generate_n_2d_bases(10)
test_results_br2d = br2d.tests_br2d(basis_list)
test_results_brlll = tests_brlll(basis_list)

for i in range(len(test_results_br2d)):
    res_2d = test_results_br2d[i]
    res_lll = test_results_brlll[i]

    b1_2d, b2_2d = res_2d["b1"], res_2d["b2"]
    b1_lll, b2_lll = res_lll["b1"], res_lll["b2"]

    match = bases_equal_2d([b1_2d,b2_2d], [b1_lll, b2_lll])

    if match:
        print(f"Test {i+1}: ✅ MATCH")
        print(f"  🔹 br2d  → b1 = {b1_2d}, b2 = {b2_2d}")
        print(f"  🔸 brlll → b1 = {b1_lll}, b2 = {b2_lll}")
    else:
        print(f"Test {i+1}: ❌ DIFFERENT")
        print(f"  🔹 br2d  → b1 = {b1_2d}, b2 = {b2_2d}")
        print(f"  🔸 brlll → b1 = {b1_lll}, b2 = {b2_lll}")




Test 1: ✅ MATCH
  🔹 br2d  → b1 = [ 26 -23], b2 = [-24 -42]
  🔸 brlll → b1 = [ 26 -23], b2 = [24 42]
Test 2: ✅ MATCH
  🔹 br2d  → b1 = [-11  44], b2 = [50 11]
  🔸 brlll → b1 = [-11  44], b2 = [50 11]
Test 3: ✅ MATCH
  🔹 br2d  → b1 = [-14  12], b2 = [37 38]
  🔸 brlll → b1 = [-14  12], b2 = [37 38]
Test 4: ✅ MATCH
  🔹 br2d  → b1 = [-11  28], b2 = [46 16]
  🔸 brlll → b1 = [-11  28], b2 = [46 16]
Test 5: ✅ MATCH
  🔹 br2d  → b1 = [-2 41], b2 = [45 10]
  🔸 brlll → b1 = [45 10], b2 = [-2 41]
Test 6: ✅ MATCH
  🔹 br2d  → b1 = [-32 -11], b2 = [-19  47]
  🔸 brlll → b1 = [-32 -11], b2 = [-19  47]
Test 7: ✅ MATCH
  🔹 br2d  → b1 = [-10   3], b2 = [ 3 27]
  🔸 brlll → b1 = [10 -3], b2 = [ 3 27]
Test 8: ✅ MATCH
  🔹 br2d  → b1 = [-16 -42], b2 = [-41  29]
  🔸 brlll → b1 = [-16 -42], b2 = [-41  29]
Test 9: ✅ MATCH
  🔹 br2d  → b1 = [21 25], b2 = [ 22 -27]
  🔸 brlll → b1 = [ 22 -27], b2 = [21 25]
Test 10: ✅ MATCH
  🔹 br2d  → b1 = [ 6 14], b2 = [-21  16]
  🔸 brlll → b1 = [ 6 14], b2 = [-21  16]
